<a target="_blank" href="https://colab.research.google.com/github/tuankhoin/MCEN90048-EEG-Grasp-and-Lift-Detection/blob/main/code.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
import os
import json
import zipfile
from pathlib import Path

try:
  # See if you are running on colab
  from google.colab import drive 

  # Download the dataset
  api_token = {"username":"tuankhoinguyen","key":"6b992b229c6a4d8f6a8b30d1c101507d"}
  kaggle_path = Path('/root/.kaggle')
  os.makedirs(kaggle_path, exist_ok=True)

  with open (kaggle_path/'kaggle.json', 'w') as handl:
      json.dump(api_token,handl)
  os.chmod(kaggle_path/'kaggle.json', 600)  
  !kaggle competitions download -c grasp-and-lift-eeg-detection

  # Unzipping datasets
  for z in ['grasp-and-lift-eeg-detection','train']:
    print(f'Unzipping {z}.zip')
    with zipfile.ZipFile(f'{z}.zip', 'r') as zip_ref:
      zip_ref.extractall('')
  print('All good!')
  # Removing the zip files to save up VM space
  !rm *.zip
except:
  pass

 99% 1.01G/1.02G [00:05<00:00, 198MB/s]
100% 1.02G/1.02G [00:05<00:00, 189MB/s]
Unzipping grasp-and-lift-eeg-detection.zip
Unzipping train.zip
All good!


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import glob
import tqdm
import pywt
import warnings
warnings.filterwarnings('ignore')

# Data observing & preprocessing

In [ ]:
datas = glob.glob('/train/*_data.csv')

In [ ]:
dir = 'train_filtered'
T=0.8
w='db2'

if not os.path.exists(dir):
  os.mkdir(dir)

for d in tqdm.tqdm(datas):
  fname = d.split('/')[-1][:-8]
  df = pd.read_csv(d).iloc[:,1:]
  out = pd.DataFrame()
  for c in df.columns:
    coeffs = pywt.wavedec(df[c],w)
    new_c = [pywt.threshold(co, T*max(co)) if i else co for i,co in enumerate(coeffs)]
    out[c] = pywt.waverec(new_c,w)
  out.to_csv(f'{dir}/{fname}_filtered.csv')